In [ ]:
# ポジ（アンカー）を入力し、ポジネガを出力

In [ ]:
import pandas as pd
import numpy as np
import itertools
import random

INPUT_PATH = "/content/drive/My Drive/NLP/work/10_my_task/input/"  
NEGA_SIZE = 7
random.seed(12345)

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
df = pd.read_csv(INPUT_PATH + "sample_1.csv", encoding='shift-jis')
print(df.shape)
df.head(3)

(11, 2)


,Q,T
0,振込限度額を教えてください。,振込限度額は100万円です。
1,１回の振込上限額はいくらですか？,振込限度額は100万円です。
2,口座開設したいです。,口座開設方法はこちらになります。


In [ ]:
df['label'] = 1
print(df.shape)
df.head(3)

(11, 3)


,Q,T,label
0,振込限度額を教えてください。,振込限度額は100万円です。,1
1,１回の振込上限額はいくらですか？,振込限度額は100万円です。,1
2,口座開設したいです。,口座開設方法はこちらになります。,1


In [ ]:
# ユニークテンプレートのセット
t_set = set(df['T'])
# print(t_set)

In [ ]:
# 
nega_q = []
nega_t = []

for index, row in df.iterrows():
  # ネガT候補
  nega_t_list = list(t_set - set({row['T']}))
  nega_t_samples = random.sample(nega_t_list, NEGA_SIZE)
  # QとネガTの組み合わせ
  for nega_t_sample in nega_t_samples:
    nega_q.append(row['Q'])
    nega_t.append(nega_t_sample)

In [ ]:
# 
df_nega = pd.DataFrame({'Q':nega_q, 'T':nega_t})
df_nega['label'] = 0
print(df_nega.shape)
df_nega.head(3)

(77, 3)


,Q,T,label
0,振込限度額を教えてください。,利払いは年１回です。,0
1,振込限度額を教えてください。,残高の確認方法はこちら。,0
2,振込限度額を教えてください。,住所変更はこちらをご覧ください。,0


In [ ]:
# 
df_posinega = pd.concat([df, df_nega], axis=0)
df_posinega = df_posinega.sort_values(by=['Q','label'], ascending=[False, False]).reset_index(drop=True)
print(df_posinega.shape)
df_posinega.head(3)

(88, 3)


,Q,T,label
0,１回の振込上限額はいくらですか？,振込限度額は100万円です。,1
1,１回の振込上限額はいくらですか？,利払いは年１回です。,0
2,１回の振込上限額はいくらですか？,口座開設方法はこちらになります。,0


In [ ]:
df_posinega.to_csv(INPUT_PATH + "posi_nega.csv", index=False)

In [ ]:
# データ分割(TODO:シャッフルなし？)
from sklearn.model_selection import train_test_split

df_trainval, df_test =  train_test_split(df_posinega, test_size=0.2, shuffle=True)  
df_train, df_val =  train_test_split(df_trainval, test_size=0.2, shuffle=True)  
print(len(df_train), len(df_val), len(df_test))

56 14 18


In [ ]:
df_train.to_csv(INPUT_PATH + "train_posi_nega.csv", index=False)
df_val.to_csv(INPUT_PATH + "val_posi_nega.csv", index=False)
df_test.to_csv(INPUT_PATH + "test_posi_nega.csv", index=False)